In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

#### Create connection between Azure databricks and Azure data storage account

In [0]:
 
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "0d5d8da8-faaf-4cf7-b0de-4852344b35a6",
"fs.azure.account.oauth2.client.secret": 'fr-8Q~qy7h-CkBnRuSlW1n9L7P1F27GJs7gcLcTQ',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/f00f2f39-2ff1-431f-aba9-3273f8ad190e/oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyo-olympics-data@tokyoolympicshahddata.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-1614657127955406> in <module> 
 7 
 8 
 ----> 9 dbutils.fs.mount(
 10 source = "abfss://tokyo-olympics-data@tokyoolympicshahddata.dfs.core.windows.net" , # contrainer@storageacc 
 11 mount_point = "/mnt/tokyoolymic" , 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o418.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyoolymic; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyoolymic
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyoolymic
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:796)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:1178)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:951)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:1167)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:804)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:128)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.$anonfun$applyOrElse$10(DbfsServerBackend.scala:461)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:273)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:269)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	

#### to see all of the files we accessed on the data lake

In [0]:
%fs
ls "/mnt/tokyoolymic"

File "<command-1614657127955407>" , line 3 
 ls "/mnt/tokyoolymic" 
 ^
 SyntaxError : invalid syntax

### check spark session

In [0]:
spark

Out[7]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

#### now for reading the data use `.read` and `.load` from the mount location, and to have the column names provided in our file will use `.option("header", "true")`

#### the `.option("inferSchema","true")` will make sure to assign the right data type in the schema of the `.csv` files

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/teams.csv")

In [0]:
athletes.printSchema()


root
-- PersonName: string (nullable = true)
-- Country: string (nullable = true)
-- Discipline: string (nullable = true)

In [0]:

coaches.printSchema()

root
-- Name: string (nullable = true)
-- Country: string (nullable = true)
-- Discipline: string (nullable = true)
-- Event: string (nullable = true)

In [0]:
entriesgender.printSchema()

root
-- Discipline: string (nullable = true)
-- Female: integer (nullable = true)
-- Male: integer (nullable = true)
-- Total: integer (nullable = true)

In [0]:
entriesgender.show()

+--------------------+------+----+-----+
 Discipline|Female|Male|Total|
+--------------------+------+----+-----+
 3x3 Basketball| 32| 32| 64|
 Archery| 64| 64| 128|
 Artistic Gymnastics| 98| 98| 196|
 Artistic Swimming| 105| 0| 105|
 Athletics| 969|1072| 2041|
 Badminton| 86| 87| 173|
 Baseball/Softball| 90| 144| 234|
 Basketball| 144| 144| 288|
 Beach Volleyball| 48| 48| 96|
 Boxing| 102| 187| 289|
 Canoe Slalom| 41| 41| 82|
 Canoe Sprint| 123| 126| 249|
Cycling BMX Frees...| 10| 9| 19|
 Cycling BMX Racing| 24| 24| 48|
Cycling Mountain ...| 38| 38| 76|
 Cycling Road| 70| 131| 201|
 Cycling Track| 90| 99| 189|
 Diving| 72| 71| 143|
 Equestrian| 73| 125| 198|
 Fencing| 107| 108| 215|
+--------------------+------+----+-----+
only showing top 20 rows

#### in the `Entries gender` table the Female and male columns are considered `string` but the values are numeric, so the data type need to be changed to `integer`


In [0]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
entriesgender.printSchema()

root
-- Discipline: string (nullable = true)
-- Female: integer (nullable = true)
-- Male: integer (nullable = true)
-- Total: integer (nullable = true)

In [0]:
medals.printSchema()

root
-- Rank: integer (nullable = true)
-- TeamCountry: string (nullable = true)
-- Gold: integer (nullable = true)
-- Silver: integer (nullable = true)
-- Bronze: integer (nullable = true)
-- Total: integer (nullable = true)
-- Rank by Total: integer (nullable = true)

In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()

+--------------------+----+
 TeamCountry|Gold|
+--------------------+----+
United States of ...| 39|
People's Republic...| 38|
 Japan| 27|
 Great Britain| 22|
 ROC| 20|
 Australia| 17|
 Netherlands| 10|
 Italy| 10|
 Germany| 10|
 France| 10|
 Canada| 7|
 New Zealand| 7|
 Brazil| 7|
 Cuba| 7|
 Hungary| 6|
 Republic of Korea| 6|
 Poland| 4|
 Czech Republic| 4|
 Kenya| 4|
 Norway| 4|
+--------------------+----+
only showing top 20 rows

In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
 Discipline|Female|Male|Total| Avg_Female| Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
 3x3 Basketball| 32| 32| 64| 0.5| 0.5|
 Archery| 64| 64| 128| 0.5| 0.5|
 Artistic Gymnastics| 98| 98| 196| 0.5| 0.5|
 Artistic Swimming| 105| 0| 105| 1.0| 0.0|
 Athletics| 969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
 Badminton| 86| 87| 173|0.49710982658959535| 0.5028901734104047|
 Baseball/Softball| 90| 144| 234|0.38461538461538464| 0.6153846153846154|
 Basketball| 144| 144| 288| 0.5| 0.5|
 Beach Volleyball| 48| 48| 96| 0.5| 0.5|
 Boxing| 102| 187| 289|0.35294117647058826| 0.6470588235294118|
 Canoe Slalom| 41| 41| 82| 0.5| 0.5|
 Canoe Sprint| 123| 126| 249| 0.4939759036144578| 0.5060240963855421|
Cycling BMX Frees...| 10| 9| 19| 0.5263157894736842|0.47368421052631576|
 Cycling BMX Racing| 24| 24| 48| 0.5| 0.5|
Cycling Mountain ...| 38| 38| 76| 0.5| 0.5|
 Cycling Road| 70| 131| 201| 0.3482587064676617| 0.6517412935323383|
 Cycling Track| 90| 99| 189|0.47619047619047616| 0.5238095238095238|
 Diving| 72| 71| 143| 0.5034965034965035| 0.4965034965034965|
 Equestrian| 73| 125| 198| 0.3686868686868687| 0.6313131313131313|
 Fencing| 107| 108| 215|0.49767441860465117| 0.5023255813953489|
+--------------------+------+----+-----+-------------------+-------------------+
only showing top 20 rows

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/transformed-data/athletes") 
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/teams")